In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install -U ginza
import pkg_resources, imp
imp.reload(pkg_resources)

     |████████████████████████████████| 54.9MB 76kB/s 
     |████████████████████████████████| 81kB 10.4MB/s 
     |████████████████████████████████| 44.8MB 97kB/s 
     |████████████████████████████████| 481kB 34.8MB/s 
  Created wheel for ginza: filename=ginza-3.1.2-cp36-none-any.whl size=17311 sha256=c8b29b1855a58190c99f2105d33ff2f0afa71878a08e32a214abd1d62b473161
  Stored in directory: /root/.cache/pip/wheels/70/8d/57/f089078acc0dbaebffc08c178e9f20924fa794c114ad36f7f7
  Created wheel for ja-ginza: filename=ja_ginza-3.1.0-cp36-none-any.whl size=54963619 sha256=60e250727ee2ecb798eb64d10d093dfc932d5871bb53b6d49d1902ffd1e4d8c6
  Stored in directory: /root/.cache/pip/wheels/f2/8a/07/1837eeb5c5648fa8d266102b78a894e495234585ac3f024cf1
  Created wheel for ja-ginza-dict: filename=ja_ginza_dict-3.1.0-cp36-none-any.whl size=70877544 sha256=afbf23966bedc1f1b92ccbd63ee3de81ebca65db66576f6ca1e02660a84fb62d
  Stored in directory: /root/.cache/pip/wheels/30/88/d7/7f0692ba26060966af34538e1079438d16

<module 'pkg_resources' from '/usr/local/lib/python3.6/dist-packages/pkg_resources/__init__.py'>

In [0]:
import numpy as np
import spacy
nlp = spacy.load('ja_ginza')

In [0]:
Dpath = '/content/drive/My Drive/Resources/KNBC_v1.0_090925_utf8/corpus2/'
topics = ['Gourmet', 'Keitai', 'Kyoto', 'Sports']

In [0]:
import pandas as pd
import re

cols = ['doc', 'label']
df = pd.DataFrame(index=[], columns=cols)
p = re.compile(r'(［.*］)(.*)')

for topic in topics:
  data = pd.read_table(Dpath+topic+'.tsv', header=None, dtype=str, usecols=[1])
  for d in data.itertuples():
    m = p.match(d[1])
    if m != None:  # first line of the document
      if d[0] != 0:
        df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)
      sent = m.groups()[1]
      doc = ''
    else:
      sent = d[1]
    #ここで形態素解析
    doc = doc + ' ' +  ' '.join([t.text for t in nlp(sent)])
  df = df.append(pd.DataFrame([doc.strip(), topic], index = cols).T)

In [6]:
df

,doc,label
0,烏丸 六角 の おかき 屋 さん 六角堂 の 前 に ある 、 蕪村 庵 と いう お店 に...,Gourmet
0,河原町 の 居酒屋 この 間 先輩 たち に つれ られ 、 河原町 の 居酒屋 へ 行っ ...,Gourmet
0,ちょっと 贅沢 ほんの ちょこっと な ん だ けど 、 贅沢 し たい とき に 何 を ...,Gourmet
0,食欲 の 秋 最近 おなか が 減っ て 仕方 あり ませ ん か 。 仕方 あり ませ ん...,Gourmet
0,季節 限定 も のっ て … １０ 月 と いえ ば 秋 本番 で ある 。 栗 や 秋刀魚...,Gourmet
...,...,...
0,クライマックス シリーズ 今年 の 阪神 は なん やっ た ん や ！ ！ 初め 弱く て...,Sports
0,京都 サンガ に つい て この ブログ の 右側 に 検索 窓 ある じゃ ない です か...,Sports
0,サッカー から 伺える ルール と の 付き合い 方 私自身 は 全く の サッカー 素人 ...,Sports
0,する か 見る か スポーツ は する 派 です か ？ し たい 見る 派 です か ？ ...,Sports


In [7]:
df.index

Int64Index([0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
            ...
            0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
           dtype='int64', length=249)

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
df_train, df_test = train_test_split(df, test_size=0.3)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.multiclass import OneVsRestClassifier
from sklearn import linear_model
from sklearn.svm import SVC
from sklearn.metrics import (roc_curve, auc, accuracy_score)
from sklearn.feature_extraction.text import CountVectorizer

svm_model = SVC(kernel='sigmoid', random_state=None)

#BOWへの変換
count = CountVectorizer()
train_X = count.fit_transform(df_train['doc']).toarray().astype(np.float64)
test_X = count.transform(df_test['doc']).toarray().astype(np.float64)


#TF-IDFベクトルへの変換　
#tf_idf = TfidfVectorizer(use_idf=True)
#train_X = tf_idf.fit_transform(df_train['doc']).toarray().astype(np.float64)
#test_X = tf_idf.transform(df_test['doc']).toarray().astype(np.float64)

In [0]:
train_labels = df_train['label']
test_labels = df_test['label']

In [43]:
#単純ベイズ、ロジステック回帰ok、オンラインSVMによって文書分類を行う

from sklearn.naive_bayes import BernoulliNB
clf = BernoulliNB()
clf.fit(train_X, train_labels)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [44]:
# ナイーブベイズでの結果
test_predict = clf.predict(test_X)
print("accuracy = "+str(accuracy_score(test_predict, test_labels)))

accuracy = 0.6933333333333334


In [45]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression() # ロジスティック回帰モデルのインスタンスを作成
lr.fit(train_X, train_labels) # ロジスティック回帰モデルの重みを学習

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [46]:
# ロジスティック回帰での結果
test_predict = lr.predict(test_X)
print("accuracy = "+str(accuracy_score(test_predict, test_labels)))

accuracy = 0.8666666666666667


In [47]:
# bow・SVCで学習
svm_model = linear_model.SGDClassifier(loss = 'hinge')
svm_model.fit(train_X, train_labels)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [48]:
# BOW・SVCでの結果
test_predict = svm_model.predict(test_X)
print("accuracy = "+str(accuracy_score(test_predict, test_labels)))

accuracy = 0.88
